# API

In [2]:
from sympy import *
from sympy.physics.quantum import *

In [2]:
n1,n2,n3=symbols('n_1,n_2,n_3')

In [3]:
g={i:Operator('\gamma_{}'.format(i)) for i in range(1,5)}

In [4]:
gb={i:Operator(r'\bar{{\gamma}}_{}'.format(i)) for i in range(1,5)}

In [5]:
c={i:Operator(r'c_{}'.format(i)) for i in range(1,5)}

In [6]:
cd={i:Dagger(Operator(r'c_{}'.format(i))) for i in range(1,5)}

In [7]:
def ea(n1,n2,n3):
    return ((1+n1)/(1-n1))**(1/4)* ((n2-I*n3)/sqrt(n2**2+n3**2))**(1/2)
    

In [8]:
def K(n1,n2,n3,op1,op2):
    expa=N(ea(n1,n2,n3))
    expma=1/expa
    return (1-n1**2)**(1/4)*( (expa+ expma)/2)- ((1-n1**2)**(1/4)*(expa-expma)/2  *I* op1 * op2 )

In [9]:
selfjoint=[(Dagger(gi),gi) for gi in g.values()]+[(Dagger(gi),gi) for gi in gb.values()]

In [10]:
squareone=[(gi**2,1) for gi in g.values()]+[(gi**2,1) for gi in gb.values()]

In [11]:
normalorder=[gi[i] for i in range(1,len(g)+1) for gi in [g, gb]]

In [12]:
normalorder2=[(normalorder[j]*normalorder[i],-normalorder[i]*normalorder[j]) for i in range(len(normalorder)) for j in range(i+1,len(normalorder))]

In [13]:
g2c=[(g[i],(c[i]+Dagger(c[i]))) for i in range(1,len(g)+1)]+[(gb[i],(Dagger(c[i])-c[i])/(I)) for i in range(1,len(gb)+1)]

In [14]:
squarezero=[(ci**2,0) for ci in c.values()] +[(Dagger(ci)**2,0) for ci in c.values()]

In [15]:
normalorder_C=[Dagger(ci) for ci in c.values()]+[ci for ci in c.values()]

In [16]:
def delta(i,j):
    return 1 if i%(len(normalorder_C)//2)==j%(len(normalorder_C)//2) else 0

In [17]:
normalorder2_C=[(normalorder_C[j]*normalorder_C[i],0 if i==j else delta(i,j)-normalorder_C[i]*normalorder_C[j]) for i in range(len(normalorder_C)) for j in range(i,len(normalorder_C))]

In [18]:
def recursiveapply(expr,subs):
    expr1=expr.subs(subs)
    while not expr1== expr:
        expr=expr1
        expr1=expr.subs(subs).expand()
    return expr1

# Op

In [24]:
Kn=(K(0,0,1,g[1],gb[1]))
# Kn=(K(0,1/sqrt(2),1/sqrt(2),gb[1],g[2]))
# Kn=K(0,.1,sqrt(1-0.1**2),gb[1],g[2])
# Kn=K(.9,0,sqrt(1-0.9**2),gb[1],g[2])

In [25]:
Kn

0.707106781186547 - 5.55111512312578e-17*I - I*(5.55111512312578e-17 - 0.707106781186547*I)*\gamma_1*\bar{\gamma}_1

In [26]:
expval=[(Dagger(ci)*ci,1) for ci in c.values()]

In [27]:
(Dagger(Kn)*Kn).expand().subs(selfjoint).subs(squareone).subs(normalorder2)

1.0 - 1.57009245868377e-16*I*\gamma_1*\bar{\gamma}_1

In [28]:
(Kn).expand()

0.707106781186547 - 5.55111512312578e-17*I - 0.707106781186547*\gamma_1*\bar{\gamma}_1 - 5.55111512312578e-17*I*\gamma_1*\bar{\gamma}_1

In [29]:
(gb[1]).subs(g2c).expand().subs(normalorder2_C).expand()

-I*Dagger(c_1) + I*c_1

In [30]:
(I* g[1]* gb[1]).subs(g2c).expand().subs(normalorder2_C).expand()

1 - 2*Dagger(c_1)*c_1

In [231]:
(Kn).expand().subs(g2c).expand().subs(normalorder2_C).expand()

0.923879532511287 - 0.38268343236509*I + 0.76536686473018*I*Dagger(c_1)*c_1

## Op

In [31]:
# Kn=(K(0.9,sqrt(1-0.9**2),0,gb[1],g[2])) *(K(0.9,sqrt(1-0.9**2),0,gb[2],g[3]))
# Kn=(K(0,1/sqrt(2),1/sqrt(2),gb[1],g[2]))
Kn=(K(0.1,sqrt(1-0.1**2)*sin(pi/4),sqrt(1-0.1**2)*cos(pi/4),gb[1],g[2]))


In [32]:
Kn

0.922721055122658 - 0.0191581945936129*I - I*(0.0462519732184832 - 0.382203575319017*I)*\bar{\gamma}_1*\gamma_2

In [52]:
recursiveapply((Dagger(Kn)*Kn).subs(g2c).expand(),normalorder2_C)

1.0 - 0.1*Dagger(c_1)*Dagger(c_2) - 0.1*Dagger(c_1)*c_2 - 0.1*Dagger(c_2)*c_1 + 0.1*c_1*c_2

In [53]:
recursiveapply((Dagger(Kn)*(I*g[1]*gb[1])*Kn).expand().subs(g2c).expand(),normalorder2_C)

0.703562363973514 - 0.703562363973515*I*Dagger(c_1)*Dagger(c_2) - 1.40712472794703*Dagger(c_1)*c_1 - 0.703562363973515*I*Dagger(c_1)*c_2 + 0.703562363973515*I*Dagger(c_2)*c_1 - 0.703562363973515*I*c_1*c_2

In [54]:
recursiveapply((Dagger(Kn)*(I*g[1]*g[2])*Kn).expand().subs(g2c).expand(),normalorder2_C)

0.703562363973515 + 0.703562363973514*I*Dagger(c_1)*Dagger(c_2) - 1.40712472794703*Dagger(c_1)*c_1 + 0.703562363973514*I*Dagger(c_1)*c_2 - 0.703562363973514*I*Dagger(c_2)*c_1 + 0.703562363973514*I*c_1*c_2

In [55]:
recursiveapply((Dagger(Kn)*(I*g[1]*gb[2])*Kn).expand().subs(g2c).expand(),normalorder2_C)

-0.1 + 1.0*Dagger(c_1)*Dagger(c_2) + 0.400000000000001*Dagger(c_1)*Dagger(c_2)*c_1*c_2 + 0.2*Dagger(c_1)*c_1 - 1.0*Dagger(c_1)*c_2 - 1.0*Dagger(c_2)*c_1 + 0.2*Dagger(c_2)*c_2 - 1.0*c_1*c_2

In [56]:
recursiveapply((Dagger(Kn)*(I*gb[1]*g[2])*Kn).expand().subs(g2c).expand(),normalorder2_C)

-0.1 + 1.0*Dagger(c_1)*Dagger(c_2) + 1.0*Dagger(c_1)*c_2 + 1.0*Dagger(c_2)*c_1 - 1.0*c_1*c_2

In [57]:
recursiveapply((Dagger(Kn)*(I*gb[1]*gb[2])*Kn).expand().subs(g2c).expand(),normalorder2_C)

-0.703562363973515 - 0.703562363973514*I*Dagger(c_1)*Dagger(c_2) + 0.703562363973515*I*Dagger(c_1)*c_2 - 0.703562363973514*I*Dagger(c_2)*c_1 + 1.40712472794703*Dagger(c_2)*c_2 - 0.703562363973514*I*c_1*c_2

In [58]:
recursiveapply((Dagger(Kn)*(I*g[2]*gb[2])*Kn).expand().subs(g2c).expand(),normalorder2_C)

0.703562363973514 - 0.703562363973515*I*Dagger(c_1)*Dagger(c_2) + 0.703562363973515*I*Dagger(c_1)*c_2 - 0.703562363973515*I*Dagger(c_2)*c_1 - 1.40712472794703*Dagger(c_2)*c_2 - 0.703562363973515*I*c_1*c_2

In [308]:
recursiveapply((Dagger(Kn)*(I*g[1]*g[3])*Kn).expand().subs(g2c).expand(),normalorder2_C)

-0.784601809837321*I*Dagger(c_1)*Dagger(c_2)*Dagger(c_3)*c_1 - 1.11022302462516e-16*I*Dagger(c_1)*Dagger(c_2)*Dagger(c_3)*c_1*c_2*c_3 + 5.55111512312578e-17*I*Dagger(c_1)*Dagger(c_2)*Dagger(c_3)*c_2 + 5.55111512312578e-17*I*Dagger(c_1)*Dagger(c_2)*Dagger(c_3)*c_3 + 0.784601809837321*I*Dagger(c_1)*Dagger(c_2)*c_1*c_3 + 2.77555756156289e-17*I*Dagger(c_1)*Dagger(c_2)*c_2*c_3 + 0.435889894354067*I*Dagger(c_1)*Dagger(c_3) - 0.784601809837321*I*Dagger(c_1)*Dagger(c_3)*c_1*c_2 - 0.784601809837321*I*Dagger(c_1)*c_1*c_2*c_3 + 0.435889894354067*I*Dagger(c_1)*c_3 + 0.392300904918661*I*Dagger(c_2)*Dagger(c_3) + 2.77555756156289e-17*I*Dagger(c_2)*Dagger(c_3)*c_1*c_2 + 5.55111512312578e-17*I*Dagger(c_2)*Dagger(c_3)*c_1*c_3 + 5.55111512312578e-17*I*Dagger(c_2)*Dagger(c_3)*c_2*c_3 + 5.55111512312578e-17*I*Dagger(c_2)*c_1 - 2.77555756156289e-17*I*Dagger(c_2)*c_1*c_2*c_3 + 5.55111512312578e-17*I*Dagger(c_2)*c_2 + 0.392300904918661*I*Dagger(c_2)*c_3 - 0.435889894354067*I*Dagger(c_3)*c_1 - 5.5511151231257

In [309]:
recursiveapply((Dagger(Kn)*(I*g[1]*gb[3])*Kn).expand().subs(g2c).expand(),normalorder2_C)

0.81 + 0.9*Dagger(c_1)*Dagger(c_2) - 1.8*Dagger(c_1)*Dagger(c_2)*Dagger(c_3)*c_1 + 6.48*Dagger(c_1)*Dagger(c_2)*Dagger(c_3)*c_1*c_2*c_3 - 1.8*Dagger(c_1)*Dagger(c_2)*Dagger(c_3)*c_3 - 3.24*Dagger(c_1)*Dagger(c_2)*c_1*c_2 - 1.8*Dagger(c_1)*Dagger(c_2)*c_1*c_3 + 1.0*Dagger(c_1)*Dagger(c_3) - 1.8*Dagger(c_1)*Dagger(c_3)*c_1*c_2 - 3.24*Dagger(c_1)*Dagger(c_3)*c_1*c_3 - 1.8*Dagger(c_1)*Dagger(c_3)*c_2*c_3 - 1.62*Dagger(c_1)*c_1 + 1.8*Dagger(c_1)*c_1*c_2*c_3 - 0.9*Dagger(c_1)*c_2 - 1.0*Dagger(c_1)*c_3 + 0.9*Dagger(c_2)*Dagger(c_3) + 2.77555756156289e-17*Dagger(c_2)*Dagger(c_3)*c_1*c_2 - 1.8*Dagger(c_2)*Dagger(c_3)*c_1*c_3 - 3.24*Dagger(c_2)*Dagger(c_3)*c_2*c_3 - 0.9*Dagger(c_2)*c_1 + 2.77555756156289e-17*Dagger(c_2)*c_1*c_2*c_3 - 1.62*Dagger(c_2)*c_2 - 0.9*Dagger(c_2)*c_3 - 1.0*Dagger(c_3)*c_1 + 1.8*Dagger(c_3)*c_1*c_2*c_3 - 0.9*Dagger(c_3)*c_2 - 1.62*Dagger(c_3)*c_3 - 0.9*c_1*c_2 - 1.0*c_1*c_3 - 0.9*c_2*c_3

In [310]:
recursiveapply((Dagger(Kn)*(I*g[2]*gb[3])*Kn).expand().subs(g2c).expand(),normalorder2_C)

0.392300904918661 + 1.38777878078145e-17*Dagger(c_1)*Dagger(c_2)*Dagger(c_3)*c_1 + 2.22044604925031e-16*Dagger(c_1)*Dagger(c_2)*Dagger(c_3)*c_1*c_2*c_3 - 2.77555756156289e-17*Dagger(c_1)*Dagger(c_2)*Dagger(c_3)*c_2 - 1.11022302462516e-16*Dagger(c_1)*Dagger(c_2)*Dagger(c_3)*c_3 - 5.55111512312578e-17*Dagger(c_1)*Dagger(c_2)*c_1*c_2 - 2.77555756156289e-17*Dagger(c_1)*c_1*c_2*c_3 + 0.435889894354067*Dagger(c_2)*Dagger(c_3) + 5.55111512312578e-17*Dagger(c_2)*Dagger(c_3)*c_1*c_2 - 5.55111512312578e-17*Dagger(c_2)*Dagger(c_3)*c_1*c_3 - 1.56920361967464*Dagger(c_2)*Dagger(c_3)*c_2*c_3 + 2.77555756156289e-17*Dagger(c_2)*c_1*c_2*c_3 - 0.784601809837321*Dagger(c_2)*c_2 - 0.435889894354067*Dagger(c_2)*c_3 + 5.55111512312578e-17*Dagger(c_3)*c_1 - 0.435889894354067*Dagger(c_3)*c_2 - 0.784601809837321*Dagger(c_3)*c_3 + 5.55111512312578e-17*c_1*c_3 - 0.435889894354067*c_2*c_3

In [313]:
recursiveapply((Dagger(Kn)*(I*gb[2]*g[3])*Kn).expand().subs(g2c).expand(),normalorder2_C)

0.9 + 0.81*Dagger(c_1)*Dagger(c_2) + 2.77555756156289e-17*Dagger(c_1)*Dagger(c_2)*Dagger(c_3)*c_1 + 1.8*Dagger(c_1)*Dagger(c_2)*Dagger(c_3)*c_2 - 2.77555756156289e-17*Dagger(c_1)*Dagger(c_2)*c_1*c_3 - 1.8*Dagger(c_1)*Dagger(c_2)*c_2*c_3 + 0.9*Dagger(c_1)*Dagger(c_3) - 1.38777878078145e-17*Dagger(c_1)*Dagger(c_3)*c_1*c_2 + 0.81*Dagger(c_1)*c_2 + 0.9*Dagger(c_1)*c_3 + 1.0*Dagger(c_2)*Dagger(c_3) - 1.8*Dagger(c_2)*Dagger(c_3)*c_1*c_2 - 5.55111512312578e-17*Dagger(c_2)*Dagger(c_3)*c_1*c_3 - 1.66533453693773e-16*Dagger(c_2)*Dagger(c_3)*c_2*c_3 + 0.81*Dagger(c_2)*c_1 - 1.8*Dagger(c_2)*c_1*c_2*c_3 + 1.0*Dagger(c_2)*c_3 + 0.9*Dagger(c_3)*c_1 - 5.55111512312578e-17*Dagger(c_3)*c_1*c_2*c_3 + 1.0*Dagger(c_3)*c_2 + 2.77555756156289e-17*Dagger(c_3)*c_3 - 0.81*c_1*c_2 - 0.9*c_1*c_3 - 1.0*c_2*c_3

In [312]:
recursiveapply((Dagger(Kn)*(I*g[3]*gb[3])*Kn).expand().subs(g2c).expand(),normalorder2_C)

0.435889894354067 + 0.392300904918661*Dagger(c_1)*Dagger(c_2) + 5.55111512312578e-17*Dagger(c_1)*Dagger(c_2)*Dagger(c_3)*c_1 + 2.77555756156289e-17*Dagger(c_1)*Dagger(c_2)*Dagger(c_3)*c_1*c_2*c_3 + 1.11022302462516e-16*Dagger(c_1)*Dagger(c_2)*Dagger(c_3)*c_2 - 0.784601809837321*Dagger(c_1)*Dagger(c_2)*Dagger(c_3)*c_3 + 2.77555756156289e-17*Dagger(c_1)*Dagger(c_2)*c_1*c_2 + 5.55111512312578e-17*Dagger(c_1)*Dagger(c_3)*c_1*c_2 + 1.38777878078145e-17*Dagger(c_1)*Dagger(c_3)*c_1*c_3 + 0.784601809837321*Dagger(c_1)*Dagger(c_3)*c_2*c_3 - 2.77555756156289e-17*Dagger(c_1)*c_1*c_2*c_3 + 0.392300904918661*Dagger(c_1)*c_2 + 2.77555756156289e-17*Dagger(c_2)*Dagger(c_3) + 0.784601809837321*Dagger(c_2)*Dagger(c_3)*c_1*c_3 + 5.55111512312578e-17*Dagger(c_2)*Dagger(c_3)*c_2*c_3 + 0.392300904918661*Dagger(c_2)*c_1 + 5.55111512312578e-17*Dagger(c_2)*c_3 + 0.784601809837321*Dagger(c_3)*c_1*c_2*c_3 + 5.55111512312578e-17*Dagger(c_3)*c_2 - 0.871779788708135*Dagger(c_3)*c_3 - 0.392300904918661*c_1*c_2

In [106]:
Kn.subs(g2c).expand().subs(squarezero).subs(normalorder2_C)

0.923879532511287 + 0.191341716182545*c_1*Dagger(c_1)

In [126]:
(Dagger(Kn)*(I*g[1]*gb[1])*Kn).expand().subs(g2c).expand().subs(normalorder2_C).subs(normalorder2_C)

0.426776695296637*I*c_1*Dagger(c_1)

In [118]:
(Dagger(Kn)*(I*g[1]*g[2])*Kn).expand().subs(g2c).expand().subs(normalorder2_C).subs(normalorder2_C).subs(normalorder2_C)

0.213388347648318*I*Dagger(c_1)*Dagger(c_2) + 0.213388347648318*I*c_1*Dagger(c_2) + 0.213388347648318*I*c_1*c_2 - 0.213388347648318*I*c_2*Dagger(c_1)

In [119]:
(Dagger(Kn)*(I*g[1]*gb[2])*Kn).expand().subs(g2c).expand().subs(normalorder2_C).subs(normalorder2_C)

0.213388347648318*I*Dagger(c_1)*Dagger(c_2) + 0.213388347648318*I*c_1*Dagger(c_2) - 0.213388347648318*I*c_1*c_2 + 0.0220970869120796*I*c_1**2*c_2*Dagger(c_1) + 0.213388347648318*I*c_2*Dagger(c_1)

In [44]:
N((1+I)**(0.5))

1.09868411346781 + 0.455089860562227*I

In [24]:
N(ea(0,1/sqrt(2),1/sqrt(2)))

0.923879532511287 - 0.38268343236509*I

In [17]:
N(exp(I*pi/8))

0.923879532511287 + 0.38268343236509*I

# Initial

In [73]:
# s=Dagger(c[1]-c[2])/sqrt(2)
s=(1-Dagger(c[1])*Dagger(c[2]))/sqrt(2)

In [74]:
recursiveapply((Dagger(s)*s).expand().subs(g2c).expand(),normalorder2_C)

1 - Dagger(c_1)*Dagger(c_2)/2 - Dagger(c_1)*Dagger(c_2)*c_1*c_2/2 - Dagger(c_1)*c_1/2 - Dagger(c_2)*c_2/2 + c_1*c_2/2

In [75]:
recursiveapply((Dagger(s)*I*g[1]*gb[1]*s).expand().subs(g2c).expand(),normalorder2_C)

Dagger(c_1)*Dagger(c_2)/2 + Dagger(c_1)*Dagger(c_2)*c_1*c_2/2 - Dagger(c_1)*c_1/2 + Dagger(c_2)*c_2/2 - c_1*c_2/2

In [76]:
recursiveapply((Dagger(s)*I*g[1]*g[2]*s).expand().subs(g2c).expand(),normalorder2_C)

I*Dagger(c_1)*Dagger(c_2)/2 + I*Dagger(c_1)*c_2/2 - I*Dagger(c_2)*c_1/2 + I*c_1*c_2/2

In [77]:
recursiveapply((Dagger(s)*I*g[1]*gb[2]*s).expand().subs(g2c).expand(),normalorder2_C)

-1 + Dagger(c_1)*Dagger(c_2)/2 + Dagger(c_1)*Dagger(c_2)*c_1*c_2 + Dagger(c_1)*c_1 - Dagger(c_1)*c_2/2 - Dagger(c_2)*c_1/2 + Dagger(c_2)*c_2 - c_1*c_2/2

In [78]:
recursiveapply((Dagger(s)*I*gb[1]*g[2]*s).expand().subs(g2c).expand(),normalorder2_C)

-1 + Dagger(c_1)*Dagger(c_2)/2 + Dagger(c_1)*Dagger(c_2)*c_1*c_2 + Dagger(c_1)*c_1 + Dagger(c_1)*c_2/2 + Dagger(c_2)*c_1/2 + Dagger(c_2)*c_2 - c_1*c_2/2

In [79]:
recursiveapply((Dagger(s)*I*gb[1]*gb[2]*s).expand().subs(g2c).expand(),normalorder2_C)

-I*Dagger(c_1)*Dagger(c_2)/2 + I*Dagger(c_1)*c_2/2 - I*Dagger(c_2)*c_1/2 - I*c_1*c_2/2

In [80]:
recursiveapply((Dagger(s)*I*g[2]*gb[2]*s).expand().subs(g2c).expand(),normalorder2_C)

Dagger(c_1)*Dagger(c_2)/2 + Dagger(c_1)*Dagger(c_2)*c_1*c_2/2 + Dagger(c_1)*c_1/2 - Dagger(c_2)*c_2/2 - c_1*c_2/2

In [56]:
s0=Dagger(c[1]-c[2])/sqrt(2)

In [57]:
recursiveapply((Dagger(s0)*s0).expand().subs(g2c).expand(),normalorder2_C)

1 - Dagger(c_1)*c_1/2 + Dagger(c_1)*c_2/2 + Dagger(c_2)*c_1/2 - Dagger(c_2)*c_2/2

In [58]:
recursiveapply((Dagger(s0)*I*g[1]*gb[1]*s0).expand().subs(g2c).expand(),normalorder2_C)

-Dagger(c_1)*Dagger(c_2)*c_1*c_2 - Dagger(c_1)*c_1/2 - Dagger(c_1)*c_2/2 - Dagger(c_2)*c_1/2 - Dagger(c_2)*c_2/2

In [ ]:
recursiveapply((Dagger(s0)*I*g[1]*g[2]*s0).expand().subs(g2c).expand(),normalorder2_C)

0

In [56]:
recursiveapply((Dagger(s0)*I*g[1]*gb[2]*s0).expand().subs(g2c).expand(),normalorder2_C)

1 - Dagger(c_1)*Dagger(c_2)*c_1*c_2 - Dagger(c_1)*c_1 - Dagger(c_2)*c_2

In [86]:
recursiveapply((g[1]*g[2]*(I*gb[1]*I*gb[2])).expand().subs(g2c).expand(),normalorder2_C)

-1 + 4*Dagger(c_1)*Dagger(c_2)*c_1*c_2 + 2*Dagger(c_1)*c_1 + 2*Dagger(c_2)*c_2

# Class A

In [3]:
f={i:Operator(r'f_{}'.format(i)) for i in range(1,5)}
fd={i:Dagger(Operator(r'f_{}'.format(i))) for i in range(1,5)}
g={i:Operator(r'g_{}'.format(i)) for i in range(1,5)}
gd={i:Dagger(Operator(r'g_{}'.format(i))) for i in range(1,5)}

In [11]:
normalorder_f=[list(op.values()) for op in [fd,gd,f,g]]

In [12]:
normalorder_f_linear=[normalorder_f[i][j] for i in range(len(normalorder_f)) for j in range(len(normalorder_f[0]))]

In [13]:
def delta(i,j):
    return 1 if (i%(len(normalorder_f_linear)//2))==(j%(len(normalorder_f_linear)//2)) else 0

In [14]:
normalorder2_f=[(normalorder_f_linear[j]*normalorder_f_linear[i],0 if i==j else delta(i,j)-normalorder_f_linear[i]*normalorder_f_linear[j]) for i in range(len(normalorder_f_linear)) for j in range(i,len(normalorder_f_linear))]

In [8]:
def recursiveapply(expr,subs):
    expr1=expr.subs(subs)
    while not expr1== expr:
        expr=expr1
        expr1=expr.subs(subs).expand()
    return expr1

In [10]:
def commutator(op1,op2,):
    return op1*op2-op2*op1

In [24]:
recursiveapply(commutator(fd[1]*f[2]+fd[2]*f[1],fd[1]*f[1]+fd[2]*f[2]).expand().subs(normalorder2_f),normalorder2_f)

0

In [25]:
recursiveapply(commutator(I*fd[1]*f[2]-I*fd[2]*f[1],fd[1]*f[1]+fd[2]*f[2]).expand().subs(normalorder2_f),normalorder2_f)

0

In [26]:
recursiveapply(commutator(fd[1]*f[1]-fd[2]*f[2],fd[1]*f[1]+fd[2]*f[2]).expand().subs(normalorder2_f),normalorder2_f)

0

In [32]:
alpha,beta,gamma=symbols('\\alpha,\\beta,\\gamma',real=True)

In [50]:
Dagger(alpha)

\alpha

In [33]:
x=alpha*(fd[1]*f[2]+fd[2]*f[1])
# x=(fd[1]*f[2]+fd[2]*f[1])

In [36]:
y=beta*(-I*fd[1]*f[2]+I*fd[2]*f[1])
# y=(-I*fd[1]*f[2]+I*fd[2]*f[1])

In [35]:
z=I*gamma*(fd[1]*f[1]-fd[2]*f[2])
# z=(fd[1]*f[1]-fd[2]*f[2])

In [41]:
xyz=(alpha-I*beta)*fd[1]*f[2]+(alpha+I*beta)*fd[2]*f[1]+I*gamma*(fd[1]*f[1]-fd[2]*f[2])

In [46]:
recursiveapply((xyz**1).expand(),normalorder2_f)

\alpha*Dagger(f_1)*f_2 + \alpha*Dagger(f_2)*f_1 - I*\beta*Dagger(f_1)*f_2 + I*\beta*Dagger(f_2)*f_1 + I*\gamma*Dagger(f_1)*f_1 - I*\gamma*Dagger(f_2)*f_2

In [45]:
recursiveapply((xyz**2).expand(),normalorder2_f)

2*\alpha**2*Dagger(f_1)*Dagger(f_2)*f_1*f_2 + \alpha**2*Dagger(f_1)*f_1 + \alpha**2*Dagger(f_2)*f_2 + 2*\beta**2*Dagger(f_1)*Dagger(f_2)*f_1*f_2 + \beta**2*Dagger(f_1)*f_1 + \beta**2*Dagger(f_2)*f_2 - 2*\gamma**2*Dagger(f_1)*Dagger(f_2)*f_1*f_2 - \gamma**2*Dagger(f_1)*f_1 - \gamma**2*Dagger(f_2)*f_2

In [47]:
recursiveapply((xyz**3).expand(),normalorder2_f)

\alpha**3*Dagger(f_1)*f_2 + \alpha**3*Dagger(f_2)*f_1 - I*\alpha**2*\beta*Dagger(f_1)*f_2 + I*\alpha**2*\beta*Dagger(f_2)*f_1 + I*\alpha**2*\gamma*Dagger(f_1)*f_1 - I*\alpha**2*\gamma*Dagger(f_2)*f_2 + \alpha*\beta**2*Dagger(f_1)*f_2 + \alpha*\beta**2*Dagger(f_2)*f_1 - \alpha*\gamma**2*Dagger(f_1)*f_2 - \alpha*\gamma**2*Dagger(f_2)*f_1 - I*\beta**3*Dagger(f_1)*f_2 + I*\beta**3*Dagger(f_2)*f_1 + I*\beta**2*\gamma*Dagger(f_1)*f_1 - I*\beta**2*\gamma*Dagger(f_2)*f_2 + I*\beta*\gamma**2*Dagger(f_1)*f_2 - I*\beta*\gamma**2*Dagger(f_2)*f_1 - I*\gamma**3*Dagger(f_1)*f_1 + I*\gamma**3*Dagger(f_2)*f_2

In [58]:
((alpha-I*beta)**3).expand()

\alpha**3 - 3*I*\alpha**2*\beta - 3*\alpha*\beta**2 + I*\beta**3

In [62]:
((alpha-1)*(alpha**2+alpha+1)).expand()

\alpha**3 - 1

In [69]:
a11,a22,a12,a21,b=symbols('a_11,a_22,a_12,a_21,b',real=True)
A11,A22,A12,A21,B=symbols('A_11,A_22,A_12,A_21,B',real=True)

In [89]:
rc=recursiveapply(((a11*fd[1]*f[1]+a22*fd[2]*f[2]+a12*fd[1]*f[2]+a21*fd[2]*f[1]+b*(fd[1]*fd[2]*f[1]*f[2]))*(A11*fd[1]*f[1]+A22*fd[2]*f[2]+A12*fd[1]*f[2]+A21*fd[2]*f[1])).expand(),normalorder2_f)

In [90]:
rc

A_11*a_11*Dagger(f_1)*f_1 + A_11*a_21*Dagger(f_2)*f_1 - A_11*a_22*Dagger(f_1)*Dagger(f_2)*f_1*f_2 + A_11*b*Dagger(f_1)*Dagger(f_2)*f_1*f_2 + A_12*a_11*Dagger(f_1)*f_2 + A_12*a_21*Dagger(f_1)*Dagger(f_2)*f_1*f_2 + A_12*a_21*Dagger(f_2)*f_2 + A_21*a_12*Dagger(f_1)*Dagger(f_2)*f_1*f_2 + A_21*a_12*Dagger(f_1)*f_1 + A_21*a_22*Dagger(f_2)*f_1 - A_22*a_11*Dagger(f_1)*Dagger(f_2)*f_1*f_2 + A_22*a_12*Dagger(f_1)*f_2 + A_22*a_22*Dagger(f_2)*f_2 + A_22*b*Dagger(f_1)*Dagger(f_2)*f_1*f_2

In [55]:
recursiveapply((xyz**4).expand(),normalorder2_f)

2*\alpha**4*Dagger(f_1)*Dagger(f_2)*f_1*f_2 + \alpha**4*Dagger(f_1)*f_1 + \alpha**4*Dagger(f_2)*f_2 + 4*\alpha**2*\beta**2*Dagger(f_1)*Dagger(f_2)*f_1*f_2 + 2*\alpha**2*\beta**2*Dagger(f_1)*f_1 + 2*\alpha**2*\beta**2*Dagger(f_2)*f_2 - 4*\alpha**2*\gamma**2*Dagger(f_1)*Dagger(f_2)*f_1*f_2 - 2*\alpha**2*\gamma**2*Dagger(f_1)*f_1 - 2*\alpha**2*\gamma**2*Dagger(f_2)*f_2 + 2*\beta**4*Dagger(f_1)*Dagger(f_2)*f_1*f_2 + \beta**4*Dagger(f_1)*f_1 + \beta**4*Dagger(f_2)*f_2 - 4*\beta**2*\gamma**2*Dagger(f_1)*Dagger(f_2)*f_1*f_2 - 2*\beta**2*\gamma**2*Dagger(f_1)*f_1 - 2*\beta**2*\gamma**2*Dagger(f_2)*f_2 + 2*\gamma**4*Dagger(f_1)*Dagger(f_2)*f_1*f_2 + \gamma**4*Dagger(f_1)*f_1 + \gamma**4*Dagger(f_2)*f_2

In [75]:
tm=Matrix([[A11,A21,0,0,0],[A12,A22,0,0,0],[0,0,A11,A21,0],[0,0,A12,A22,0],[-A22,A21,A12,-A11,A11+A22]])

In [76]:
init=Matrix([[a11,a12,a21,a22,b]]).T

In [92]:
((tm@init).dot(basis)-rc).expand()

0

In [119]:
rho,phi=symbols('\\rho,\\phi',real=True)

In [120]:
woU_init=[(a11,0),(a22,0),(a12,rho*exp(-I*phi)),(a21,rho*exp(I*phi)),(b,0)]
woU_M=[(A11,0),(A22,0),(A12,rho*exp(-I*phi)),(A21,rho*exp(I*phi))]
wU_init=[(a11,I*gamma),(a22,-I*gamma),(a12,rho*exp(-I*phi)),(a21,rho*exp(I*phi)),(b,0)]
wU_M=[(A11,I*gamma),(A22,-I*gamma),(A12,rho*exp(-I*phi)),(A21,rho*exp(I*phi))]

In [124]:
(init).subs(woU_init+woU_M)

Matrix([
[                0],
[\rho*exp(-I*\phi)],
[ \rho*exp(I*\phi)],
[                0],
[                0]])

In [121]:
(tm@init).subs(woU_init+woU_M)

Matrix([
[  \rho**2],
[        0],
[        0],
[  \rho**2],
[2*\rho**2]])

In [122]:
(tm@tm@init).subs(woU_init+woU_M)

Matrix([
[                   0],
[\rho**3*exp(-I*\phi)],
[ \rho**3*exp(I*\phi)],
[                   0],
[                   0]])

In [123]:
(tm**3@init).subs(woU_init+woU_M)

Matrix([
[  \rho**4],
[        0],
[        0],
[  \rho**4],
[2*\rho**4]])

In [101]:
(tm**4@init).subs(woU_init+woU_M)

Matrix([
[                                          0],
[(\alpha - I*\beta)**3*(\alpha + I*\beta)**2],
[(\alpha - I*\beta)**2*(\alpha + I*\beta)**3],
[                                          0],
[                                          0]])

In [131]:
s1=init.subs(wU_init+wU_M)

In [132]:
s1

Matrix([
[         I*\gamma],
[\rho*exp(-I*\phi)],
[ \rho*exp(I*\phi)],
[        -I*\gamma],
[                0]])

In [133]:
s2=(tm@s1).subs(wU_init+wU_M)

In [134]:
s2

Matrix([
[    -\gamma**2 + \rho**2],
[                       0],
[                       0],
[    -\gamma**2 + \rho**2],
[-2*\gamma**2 + 2*\rho**2]])

In [136]:
s3=(tm@s2).subs(wU_init+wU_M)

In [137]:
s3

Matrix([
[         I*\gamma*(-\gamma**2 + \rho**2)],
[\rho*(-\gamma**2 + \rho**2)*exp(-I*\phi)],
[ \rho*(-\gamma**2 + \rho**2)*exp(I*\phi)],
[        -I*\gamma*(-\gamma**2 + \rho**2)],
[                                       0]])

In [138]:
s4=((tm@s3).subs(wU_init+wU_M))

In [142]:
factor(s4)

Matrix([
[    -\gamma**2*(-\gamma**2 + \rho**2) + \rho**2*(-\gamma**2 + \rho**2)],
[                                                                     0],
[                                                                     0],
[    -\gamma**2*(-\gamma**2 + \rho**2) + \rho**2*(-\gamma**2 + \rho**2)],
[-2*\gamma**2*(-\gamma**2 + \rho**2) + 2*\rho**2*(-\gamma**2 + \rho**2)]])

In [140]:
s5=((tm@s4).subs(wU_init+wU_M))

In [141]:
s5

Matrix([
[         I*\gamma*(-\gamma**2*(-\gamma**2 + \rho**2) + \rho**2*(-\gamma**2 + \rho**2))],
[\rho*(-\gamma**2*(-\gamma**2 + \rho**2) + \rho**2*(-\gamma**2 + \rho**2))*exp(-I*\phi)],
[ \rho*(-\gamma**2*(-\gamma**2 + \rho**2) + \rho**2*(-\gamma**2 + \rho**2))*exp(I*\phi)],
[        -I*\gamma*(-\gamma**2*(-\gamma**2 + \rho**2) + \rho**2*(-\gamma**2 + \rho**2))],
[                                                                                     0]])

In [149]:
abs(1+I)

sqrt(2)

In [173]:
Ap=sqrt(gamma**2+2*gamma*im(sqrt(rho**2-gamma**2))+abs(rho**2-gamma**2)+rho**2)
Am=sqrt(gamma**2-2*gamma*im(sqrt(rho**2-gamma**2))+abs(rho**2-gamma**2)+rho**2)

In [174]:
Ap

sqrt(\gamma**2 + 2*\gamma*sin(atan2(0, -\gamma**2 + \rho**2)/2)*sqrt(Abs(\gamma**2 - \rho**2)) + \rho**2 + Abs(\gamma**2 - \rho**2))

In [175]:
Am

sqrt(\gamma**2 - 2*\gamma*sin(atan2(0, -\gamma**2 + \rho**2)/2)*sqrt(Abs(\gamma**2 - \rho**2)) + \rho**2 + Abs(\gamma**2 - \rho**2))

In [198]:
theta=symbols('\\theta',real=True)

In [205]:
cp=1/sqrt(2)*(exp(-I*phi)*(sech(theta)+I*tanh(theta))*f[1]+f[2])
cp_d=1/sqrt(2)*(exp(I*phi)*(sech(theta)-I*tanh(theta))*fd[1]+fd[2])
cm=1/sqrt(2)*(exp(-I*phi)*(sech(theta)-I*tanh(theta))*f[1]+f[2])
cm_d=1/sqrt(2)*(exp(I*phi)*(sech(theta)+I*tanh(theta))*fd[1]+fd[2])

In [207]:
recursiveapply((cp_d*cp-cm_d*cm).expand(),normalorder2_f)

-I*exp(I*\phi)*tanh(\theta)*Dagger(f_1)*f_2 + I*exp(-I*\phi)*tanh(\theta)*Dagger(f_2)*f_1

In [206]:
(cp_d*cp-cm_d*cm).expand()

-I*exp(I*\phi)*tanh(\theta)*Dagger(f_1)*f_2 + I*exp(-I*\phi)*tanh(\theta)*Dagger(f_2)*f_1

In [ ]:
recursiveapply((cp_d*cp-cm_d*cm).expand(),normalorder2_f)

In [211]:
eye(2)

Matrix([
[1, 0],
[0, 1]])

In [212]:
pm={0:eye(2),1:Matrix([[0,1],[1,0]]),2:Matrix([[0,-I],[I,0]]),3:Matrix([[1,0],[0,-1]])}

In [216]:
TensorProduct(pm[1],pm[1])

Matrix([
[0, 0, 0, 1],
[0, 0, 1, 0],
[0, 1, 0, 0],
[1, 0, 0, 0]])

In [218]:
Matrix([fd[1],fd[2],gd[1],gd[2]]).T@TensorProduct(pm[1],pm[0])@Matrix([f[1],f[2],g[1],g[2]])

Matrix([[Dagger(f_1)*g_1 + Dagger(f_2)*g_2 + Dagger(g_1)*f_1 + Dagger(g_2)*f_2]])

In [220]:
Matrix([fd[1],fd[2],gd[1],gd[2]]).T@TensorProduct(pm[3],pm[0])@Matrix([f[1],f[2],g[1],g[2]])

Matrix([[Dagger(f_1)*f_1 + Dagger(f_2)*f_2 - Dagger(g_1)*g_1 - Dagger(g_2)*g_2]])

In [234]:
a = {(i, j): symbols(f'a_{{{i}{j}}}',real=True) for i in range(4) for j in range(4)}


In [233]:
a[0,3]

a_{03}

In [253]:
coef={(i,j):a[i,j] if j in [1,2] else I*a[i,j] for i in range(4) for j in range(4)}

In [254]:
coef

{(0, 0): I*a_{00},
 (0, 1): a_{01},
 (0, 2): a_{02},
 (0, 3): I*a_{03},
 (1, 0): I*a_{10},
 (1, 1): a_{11},
 (1, 2): a_{12},
 (1, 3): I*a_{13},
 (2, 0): I*a_{20},
 (2, 1): a_{21},
 (2, 2): a_{22},
 (2, 3): I*a_{23},
 (3, 0): I*a_{30},
 (3, 1): a_{31},
 (3, 2): a_{32},
 (3, 3): I*a_{33}}

In [255]:
exp_herm=[coef[i,j]*(Matrix([fd[1],fd[2],gd[1],gd[2]]).T@TensorProduct(pm[i],pm[j])@Matrix([f[1],f[2],g[1],g[2]]))[0,0] for i in range(4) for j in range(4)]

In [267]:
exp_herm_simp=simplify(sum(exp_herm).expand())

In [268]:
exp_herm_simp.free_symbols

{a_{00},
 a_{01},
 a_{02},
 a_{03},
 a_{10},
 a_{11},
 a_{12},
 a_{13},
 a_{20},
 a_{21},
 a_{22},
 a_{23},
 a_{30},
 a_{31},
 a_{32},
 a_{33},
 f_1,
 f_2,
 g_1,
 g_2}

In [263]:
exp_herm[7]

I*a_{13}*(Dagger(f_1)*g_1 - Dagger(f_2)*g_2 + Dagger(g_1)*f_1 - Dagger(g_2)*f_2)

In [186]:
cp=(exp(-I*phi)*(I*gamma+sqrt(rho**2-gamma**2))*f[1]+rho*f[2])
cp_d=(exp(I*phi)*(-I*gamma+conjugate(sqrt(rho**2-gamma**2)))*fd[1]+rho*fd[2])
cm=(exp(-I*phi)*(I*gamma-sqrt(rho**2-gamma**2))*f[1]+rho*f[2])
cm_d=(exp(I*phi)*(-I*gamma-conjugate(sqrt(rho**2-gamma**2)))*fd[1]+rho*fd[2])

In [176]:
cp=1/Ap*(exp(-I*phi)*(I*gamma+sqrt(rho**2-gamma**2))*f[1]+rho*f[2])
cp_d=1/Ap*(exp(I*phi)*(-I*gamma+conjugate(sqrt(rho**2-gamma**2)))*fd[1]+rho*fd[2])
cm=1/Am*(exp(-I*phi)*(I*gamma-sqrt(rho**2-gamma**2))*f[1]+rho*f[2])
cm_d=1/Am*(exp(I*phi)*(-I*gamma-conjugate(sqrt(rho**2-gamma**2)))*fd[1]+rho*fd[2])

In [187]:
cp

\rho*f_2 + (I*\gamma + sqrt(-\gamma**2 + \rho**2))*exp(-I*\phi)*f_1

In [188]:
cp_d

\rho*Dagger(f_2) + (-I*\gamma + conjugate(sqrt(-\gamma**2 + \rho**2)))*exp(I*\phi)*Dagger(f_1)

In [189]:
cm

\rho*f_2 + (I*\gamma - sqrt(-\gamma**2 + \rho**2))*exp(-I*\phi)*f_1

In [190]:
cm_d

\rho*Dagger(f_2) + (-I*\gamma - conjugate(sqrt(-\gamma**2 + \rho**2)))*exp(I*\phi)*Dagger(f_1)

In [191]:
lp=sqrt(rho**2-gamma**2)
lm=-sqrt(rho**2-gamma**2)

In [194]:
recursiveapply((cp_d*cp).expand(),normalorder2_f)

\gamma**2*Dagger(f_1)*f_1 - I*\gamma*\rho*exp(I*\phi)*Dagger(f_1)*f_2 + I*\gamma*\rho*exp(-I*\phi)*Dagger(f_2)*f_1 - I*\gamma*sqrt(-\gamma**2 + \rho**2)*Dagger(f_1)*f_1 + I*\gamma*conjugate(sqrt(-\gamma**2 + \rho**2))*Dagger(f_1)*f_1 + \rho**2*Dagger(f_2)*f_2 + \rho*sqrt(-\gamma**2 + \rho**2)*exp(-I*\phi)*Dagger(f_2)*f_1 + \rho*exp(I*\phi)*conjugate(sqrt(-\gamma**2 + \rho**2))*Dagger(f_1)*f_2 + sqrt(-\gamma**2 + \rho**2)*conjugate(sqrt(-\gamma**2 + \rho**2))*Dagger(f_1)*f_1

In [195]:
recursiveapply(commutator(cp_d*cp,cm_d*cm).expand(),normalorder2_f)

-2*\gamma**2*\rho*sqrt(-\gamma**2 + \rho**2)*exp(I*\phi)*Dagger(f_1)*f_2 + 2*I*\gamma*\rho**2*sqrt(-\gamma**2 + \rho**2)*Dagger(f_1)*f_1 - 2*I*\gamma*\rho**2*sqrt(-\gamma**2 + \rho**2)*Dagger(f_2)*f_2 + 2*I*\gamma*\rho**2*conjugate(sqrt(-\gamma**2 + \rho**2))*Dagger(f_1)*f_1 - 2*I*\gamma*\rho**2*conjugate(sqrt(-\gamma**2 + \rho**2))*Dagger(f_2)*f_2 - 2*\rho**3*sqrt(-\gamma**2 + \rho**2)*exp(-I*\phi)*Dagger(f_2)*f_1 + 2*\rho**3*exp(I*\phi)*conjugate(sqrt(-\gamma**2 + \rho**2))*Dagger(f_1)*f_2 + 2*\rho**3*exp(-I*\phi)*conjugate(sqrt(-\gamma**2 + \rho**2))*Dagger(f_2)*f_1 - 2*\rho*sqrt(-\gamma**2 + \rho**2)*exp(I*\phi)*conjugate(sqrt(-\gamma**2 + \rho**2))**2*Dagger(f_1)*f_2

In [182]:
lp*cp_d*cp+lm*cm_d*cm

-sqrt(-\gamma**2 + \rho**2)*(\rho*Dagger(f_2) + (-I*\gamma - conjugate(sqrt(-\gamma**2 + \rho**2)))*exp(I*\phi)*Dagger(f_1))*(\rho*f_2 + (I*\gamma - sqrt(-\gamma**2 + \rho**2))*exp(-I*\phi)*f_1)/(\gamma**2 + 2*\gamma*sin(atan2(0, -\gamma**2 + \rho**2)/2)*sqrt(Abs(\gamma**2 - \rho**2)) + \rho**2 + Abs(\gamma**2 - \rho**2)) + sqrt(-\gamma**2 + \rho**2)*(\rho*Dagger(f_2) + (-I*\gamma + conjugate(sqrt(-\gamma**2 + \rho**2)))*exp(I*\phi)*Dagger(f_1))*(\rho*f_2 + (I*\gamma + sqrt(-\gamma**2 + \rho**2))*exp(-I*\phi)*f_1)/(\gamma**2 + 2*\gamma*sin(atan2(0, -\gamma**2 + \rho**2)/2)*sqrt(Abs(\gamma**2 - \rho**2)) + \rho**2 + Abs(\gamma**2 - \rho**2))

In [183]:
recursiveapply((lp*cp_d*cp+lm*cm_d*cm).expand(),normalorder2_f)

2*I*\gamma**3*Dagger(f_1)*f_1/(\gamma**2 + 2*\gamma*sin(atan2(0, -\gamma**2 + \rho**2)/2)*sqrt(Abs(\gamma**2 - \rho**2)) + \rho**2 + Abs(\gamma**2 - \rho**2)) - 2*\gamma**2*\rho*Dagger(f_2)*f_1/(\gamma**2*exp(I*\phi) + 2*\gamma*exp(I*\phi)*sin(atan2(0, -\gamma**2 + \rho**2)/2)*sqrt(Abs(\gamma**2 - \rho**2)) + \rho**2*exp(I*\phi) + exp(I*\phi)*Abs(\gamma**2 - \rho**2)) - 2*I*\gamma*\rho**2*Dagger(f_1)*f_1/(\gamma**2 + 2*\gamma*sin(atan2(0, -\gamma**2 + \rho**2)/2)*sqrt(Abs(\gamma**2 - \rho**2)) + \rho**2 + Abs(\gamma**2 - \rho**2)) + 2*I*\gamma*sqrt(-\gamma**2 + \rho**2)*conjugate(sqrt(-\gamma**2 + \rho**2))*Dagger(f_1)*f_1/(\gamma**2 + 2*\gamma*sin(atan2(0, -\gamma**2 + \rho**2)/2)*sqrt(Abs(\gamma**2 - \rho**2)) + \rho**2 + Abs(\gamma**2 - \rho**2)) + 2*\rho**3*Dagger(f_2)*f_1/(\gamma**2*exp(I*\phi) + 2*\gamma*exp(I*\phi)*sin(atan2(0, -\gamma**2 + \rho**2)/2)*sqrt(Abs(\gamma**2 - \rho**2)) + \rho**2*exp(I*\phi) + exp(I*\phi)*Abs(\gamma**2 - \rho**2)) + 2*\rho*sqrt(-\gamma**2 + \rho**2)

In [84]:
basis=Matrix([(fd[1]*f[1]),(fd[1]*f[2]),(fd[2]*f[1]),(fd[2]*f[2]),(fd[1]*fd[2]*f[1]*f[2])])

In [85]:
basis

Matrix([
[                Dagger(f_1)*f_1],
[                Dagger(f_1)*f_2],
[                Dagger(f_2)*f_1],
[                Dagger(f_2)*f_2],
[Dagger(f_1)*Dagger(f_2)*f_1*f_2]])

In [106]:
phi=Symbol(r'\phi',real=True)

In [110]:
fp_d=1/sqrt(2)*(fd[1]+exp(-I*phi)*fd[2])
fp=1/sqrt(2)*(f[1]+exp(I*phi)*f[2])
fm_d=1/sqrt(2)*(fd[1]-exp(-I*phi)*fd[2])
fm=1/sqrt(2)*(f[1]-exp(I*phi)*f[2])

In [118]:
(fp_d*fp+fm_d*fm).expand()

Dagger(f_1)*f_1 + Dagger(f_2)*f_2

In [114]:
recursiveapply(commutator(fp_d*fp,fm_d*fm).expand(),normalorder2_f)

0

In [117]:
recursiveapply(((fp_d*fp)**2-(fp_d*fp)).expand(),normalorder2_f)

0

In [48]:
def group_noncommutative(expr, op):
    # Ensure the expression is expanded so we can iterate through terms
    expr = expr.expand()
    
    # Separate the terms involving the operator `op`
    op_terms = [term for term in expr.args if op in term.free_symbols]
    
    # Separate the terms not involving the operator `op`
    other_terms = [term for term in expr.args if op not in term.free_symbols]
    
    # Sum the coefficients of the terms involving `op`
    op_coeff = sum(term.coeff(op) for term in op_terms)
    
    # Reconstruct the grouped expression
    grouped_expr = op_coeff*op + Add(*other_terms)
    
    return grouped_expr

In [54]:
group_noncommutative(alpha*f[1]+beta*f[1]+ gamma*Dagger(f[1]), f[1])

(\alpha + \beta)*f_1

In [24]:
recursiveapply(commutator(x,y).expand()-2*I*z,normalorder2_f)

0

In [25]:
recursiveapply(commutator(y,z).expand()-2*I*x,normalorder2_f)

0

In [27]:
recursiveapply(commutator(z,x).expand()-2*I*y,normalorder2_f)

0

In [22]:
z

Dagger(f_1)*f_1 - Dagger(f_2)*f_2

In [75]:
xy=recursiveapply(commutator(x+y+z,x+y-z).expand(),normalorder2_f)

In [77]:
xy

4*I*\alpha*\gamma*Dagger(f_1)*f_2 - 4*I*\alpha*\gamma*Dagger(f_2)*f_1 - 4*\beta*\gamma*Dagger(f_1)*f_2 - 4*\beta*\gamma*Dagger(f_2)*f_1

In [79]:
(x+y + (xy/2).simplify())

\alpha*(Dagger(f_1)*f_2 + Dagger(f_2)*f_1) + \beta*(I*Dagger(f_1)*f_2 - I*Dagger(f_2)*f_1) + 2*\gamma*(I*\alpha*Dagger(f_1)*f_2 - I*\alpha*Dagger(f_2)*f_1 - \beta*Dagger(f_1)*f_2 - \beta*Dagger(f_2)*f_1)

In [80]:
xy3=recursiveapply(commutator(x-y,xy).expand(),normalorder2_f)

In [81]:
xy3

-8*I*\alpha**2*\gamma*Dagger(f_1)*f_1 + 8*I*\alpha**2*\gamma*Dagger(f_2)*f_2 + 8*I*\beta**2*\gamma*Dagger(f_1)*f_1 - 8*I*\beta**2*\gamma*Dagger(f_2)*f_2

In [62]:
x+y

\alpha*(Dagger(f_1)*f_2 + Dagger(f_2)*f_1) + \beta*(I*Dagger(f_1)*f_2 - I*Dagger(f_2)*f_1)

In [63]:
xy

4*\alpha*\gamma*Dagger(f_1)*f_2 - 4*\alpha*\gamma*Dagger(f_2)*f_1 + 4*I*\beta*\gamma*Dagger(f_1)*f_2 + 4*I*\beta*\gamma*Dagger(f_2)*f_1

In [ ]:
x.subs()

In [71]:
recursiveapply(commutator(x,y).expand(),normalorder2_f)

-2*I*\alpha*\beta*Dagger(f_1)*f_1 + 2*I*\alpha*\beta*Dagger(f_2)*f_2

In [73]:
z

\gamma*(Dagger(f_1)*f_1 - Dagger(f_2)*f_2)

In [ ]:
recursiveapply(commutator(fd[1]*f[1]-fd[2]*f[2],fd[1]*f[1]+fd[2]*f[2]).expand().subs(normalorder2_f),normalorder2_f)

In [10]:
recursiveapply(((fd[1]+gd[1])*f[1]*fd[1]*g[1]*gd[1]*(f[1]+g[1])).expand(),normalorder2_f)

2*Dagger(f_1)*Dagger(g_1)*f_1*g_1 + Dagger(f_1)*f_1 + Dagger(f_1)*g_1 + Dagger(g_1)*f_1 + Dagger(g_1)*g_1

In [11]:
recursiveapply(((fd[1]-gd[1])*f[1]*fd[1]*g[1]*gd[1]*(f[1]-g[1])).expand(),normalorder2_f)

2*Dagger(f_1)*Dagger(g_1)*f_1*g_1 + Dagger(f_1)*f_1 - Dagger(f_1)*g_1 - Dagger(g_1)*f_1 + Dagger(g_1)*g_1

In [56]:
recursiveapply(((1+fd[1]*gd[1])*f[1]*fd[1]*g[1]*gd[1]*(1+g[1]*f[1])).expand(),normalorder2_f)

1 + Dagger(f_1)*Dagger(g_1) - 2*Dagger(f_1)*Dagger(g_1)*f_1*g_1 - Dagger(f_1)*f_1 - Dagger(g_1)*g_1 - f_1*g_1

In [90]:
recursiveapply(((1-fd[1]*gd[1])*f[1]*fd[1]*g[1]*gd[1]*(1-g[1]*f[1])).expand(),normalorder2_f)

1 - Dagger(f_1)*Dagger(g_1) - 2*Dagger(f_1)*Dagger(g_1)*f_1*g_1 - Dagger(f_1)*f_1 - Dagger(g_1)*g_1 + f_1*g_1

In [58]:
recursiveapply((gd[1]*f[1]+fd[1]*g[1]).expand(),normalorder2_f)

Dagger(f_1)*g_1 + Dagger(g_1)*f_1

In [61]:
recursiveapply(((gd[1]*f[1]+fd[1]*g[1])**2).expand(),normalorder2_f)

2*Dagger(f_1)*Dagger(g_1)*f_1*g_1 + Dagger(f_1)*f_1 + Dagger(g_1)*g_1

In [62]:
recursiveapply(((gd[1]*f[1]+fd[1]*g[1])**3).expand(),normalorder2_f)

Dagger(f_1)*g_1 + Dagger(g_1)*f_1

In [63]:
recursiveapply(((gd[1]*f[1]+fd[1]*g[1])**4).expand(),normalorder2_f)

2*Dagger(f_1)*Dagger(g_1)*f_1*g_1 + Dagger(f_1)*f_1 + Dagger(g_1)*g_1

In [59]:
((gd[1]*f[1]+fd[1]*g[1])**2).expand()

Dagger(f_1)*g_1*Dagger(f_1)*g_1 + Dagger(f_1)*g_1*Dagger(g_1)*f_1 + Dagger(g_1)*f_1*Dagger(f_1)*g_1 + Dagger(g_1)*f_1*Dagger(g_1)*f_1

In [60]:
((gd[1]*f[1]+fd[1]*g[1])**3).expand()

Dagger(f_1)*g_1*Dagger(f_1)*g_1*Dagger(f_1)*g_1 + Dagger(f_1)*g_1*Dagger(f_1)*g_1*Dagger(g_1)*f_1 + Dagger(f_1)*g_1*Dagger(g_1)*f_1*Dagger(f_1)*g_1 + Dagger(f_1)*g_1*Dagger(g_1)*f_1*Dagger(g_1)*f_1 + Dagger(g_1)*f_1*Dagger(f_1)*g_1*Dagger(f_1)*g_1 + Dagger(g_1)*f_1*Dagger(f_1)*g_1*Dagger(g_1)*f_1 + Dagger(g_1)*f_1*Dagger(g_1)*f_1*Dagger(f_1)*g_1 + Dagger(g_1)*f_1*Dagger(g_1)*f_1*Dagger(g_1)*f_1

In [65]:
recursiveapply(((-I*gd[1]*f[1]+I*fd[1]*g[1])*(gd[1]*f[1]+fd[1]*g[1])- (gd[1]*f[1]+fd[1]*g[1])*(-I*gd[1]*f[1]+I*fd[1]*g[1])).expand(),normalorder2_f)

2*I*Dagger(f_1)*f_1 - 2*I*Dagger(g_1)*g_1

In [66]:
recursiveapply((-I*gd[1]*f[1]+I*fd[1]*g[1]).expand(),normalorder2_f)

I*Dagger(f_1)*g_1 - I*Dagger(g_1)*f_1

In [67]:
recursiveapply(((-I*gd[1]*f[1]+I*fd[1]*g[1])**2).expand(),normalorder2_f)

2*Dagger(f_1)*Dagger(g_1)*f_1*g_1 + Dagger(f_1)*f_1 + Dagger(g_1)*g_1

In [68]:
recursiveapply(((-I*gd[1]*f[1]+I*fd[1]*g[1])**3).expand(),normalorder2_f)

I*Dagger(f_1)*g_1 - I*Dagger(g_1)*f_1

In [69]:
recursiveapply(((-I*gd[1]*f[1]+I*fd[1]*g[1])**4).expand(),normalorder2_f)

2*Dagger(f_1)*Dagger(g_1)*f_1*g_1 + Dagger(f_1)*f_1 + Dagger(g_1)*g_1

In [73]:
alpha,beta,=symbols(r'\alpha,\beta')

In [74]:
alpha

\alpha

In [81]:
exp1=(1+(cosh(alpha)-1)*(2*fd[1]*gd[1]*f[1]*g[1]+fd[1]*f[1]+gd[1]*g[1])+sinh(alpha)*(fd[1]*g[1]+gd[1]*f[1]))

In [82]:
exp2=(1+(cosh(beta)-1)*(2*fd[1]*gd[1]*f[1]*g[1]+fd[1]*f[1]+gd[1]*g[1])+sinh(beta)*(I*fd[1]*g[1]-I*gd[1]*f[1]))

In [87]:
recursiveapply((exp1*exp2).expand(),normalorder2_f)

-I*sinh(\alpha)*sinh(\beta)*Dagger(f_1)*f_1 + I*sinh(\alpha)*sinh(\beta)*Dagger(g_1)*g_1 + sinh(\alpha)*cosh(\beta)*Dagger(f_1)*g_1 + sinh(\alpha)*cosh(\beta)*Dagger(g_1)*f_1 + I*sinh(\beta)*cosh(\alpha)*Dagger(f_1)*g_1 - I*sinh(\beta)*cosh(\alpha)*Dagger(g_1)*f_1 + 2*cosh(\alpha)*cosh(\beta)*Dagger(f_1)*Dagger(g_1)*f_1*g_1 + cosh(\alpha)*cosh(\beta)*Dagger(f_1)*f_1 + cosh(\alpha)*cosh(\beta)*Dagger(g_1)*g_1 + 1 - 2*Dagger(f_1)*Dagger(g_1)*f_1*g_1 - Dagger(f_1)*f_1 - Dagger(g_1)*g_1

In [88]:
recursiveapply((exp1*exp2).expand(),normalorder2_f).simplify()

sinh(\alpha)*cosh(\beta)*Dagger(f_1)*g_1 + sinh(\alpha)*cosh(\beta)*Dagger(g_1)*f_1 - I*sinh(\beta)*(sinh(\alpha)*Dagger(f_1)*f_1 - sinh(\alpha)*Dagger(g_1)*g_1 - cosh(\alpha)*Dagger(f_1)*g_1 + cosh(\alpha)*Dagger(g_1)*f_1) + 2*cosh(\alpha)*cosh(\beta)*Dagger(f_1)*Dagger(g_1)*f_1*g_1 + cosh(\alpha)*cosh(\beta)*Dagger(f_1)*f_1 + cosh(\alpha)*cosh(\beta)*Dagger(g_1)*g_1 + 1 - 2*Dagger(f_1)*Dagger(g_1)*f_1*g_1 - Dagger(f_1)*f_1 - Dagger(g_1)*g_1